In [3]:
gamma, p = [0.1, 0.92]

In [4]:
half_life(gamma,p)

9.0446

Second approach

In [9]:
def half_life(gamma, p):
    num_wires = 1

    dev = qml.device("default.mixed", wires=num_wires)

    @qml.qnode(dev)
    def noise(gamma,p,dt,n):
        qml.Hadamard(wires=0)
        for _ in range(n):
            qml.GeneralizedAmplitudeDamping(gamma* dt,p,wires=0 )
        return qml.probs(wires=0)
    dt = 0.001
    tmin =0
    tmax = 100
    while(tmax-tmin)> dt:
        tmid = (tmax+tmin)/2
        prob = noise(gamma, p, dt, int(tmid/dt))[1]
        if prob > 1/4:
            tmin = tmid
        else:
           tmax=tmid

    return tmid

In [10]:
half_life(gamma, p)

9.044647216796875

# The chalet of the random gate
## Challenge statement
The Tensor Tundra's environment is so cold, it's an excellent place to build quantum devices. A safe haven for those who venture into these icy plains is the Chalet of the Random Gate.

The front entrance of the chalet is a quantum gate, and its action will take us to different entry points: if it's an X gate, it takes us through the backdoor, if it's a Y gate, it takes us through the chimney, and if it's a Z gate, we just go through the front door.

This gate is a programmable device, so that we can choose where to end up (for some reason, people tend to avoid the Y gate). But when a strong blizzard strikes, the controls become frozen solid. Instead of being stuck in one setting, the noise coming from the blizzard makes the gate act randomly.

In this challenge, we explore what happens when the application of a quantum gate is not done deterministically. Suppose that we have a single qubit |0⟩ on which a single gate acts at random. This gate can be:
* The Pauli X gate, acting with probability p,
* The Pauli Y gate, acting with probability q,
* The Pauli Z gate, acting with probability r,
* The Identity I, acting with probability 1−p−q−r.

You are asked to calculate the measurement probabilities in the computational basis upon the application of these random gates.

Fingers crossed that you don't get stuck in the chimney!
## Challenge Code
In the challenge template, you must complete the random_gate QNode which, given the probabilities p, q, and r (float) return the measurement probabilities (np.array(float)) in the computational basis.

## Input
As an input to this challenge, you are given the probabilities p, q and r (float) representing the application probabilities of the Pauli X, Y, and Z gates respectively.

## Output
The output is an np.tensor$[p_0,p_1]$ containing the probabilities of measuring 0 or 1 in the computational basis after the random application of the gate.

Test cases
The following public test cases are available for you to check your work. There are also some hidden test cases which we will use to check that your solution works in full generality.

test_input: [0.25,0.25,0.25]
expected_ouput: [0.5,0.5]

test_input: [0.125,0.25,0.2]
expected_ouput: [0.625, 0.375]


In [88]:
import json
import pennylane as qml
import pennylane.numpy as np
def half_life(gamma, p):
    """Calculates the relaxation half-life of a quantum system that exchanges energy with its environment.
    This process is modeled via Generalized Amplitude Damping.

    Args:
        gamma (float):
            The probability per unit time of the system losing a quantum of energy
            to the environment.
        p (float): The de-excitation probability due to environmental effect

    Returns:
        (float): The relaxation haf-life of the system, as explained in the problem statement.
    """

    num_wires = 1

    dev = qml.device("default.mixed", wires=num_wires)

    # Put your code here

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:

    ins = json.loads(test_case_input)
    output = half_life(*ins)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, atol=2e-1
    ), "The relaxation half-life is not quite right."

# These are the public test cases
test_cases = [
    ('[0.1,0.92]', '9.05'),
    ('[0.2,0.83]', '7.09')
]
# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[0.25, 0.25, 0.25]'...
Correct!
Running test case 1 with input '[0.125, 0.25, 0.2]'...
Correct!


Solution based on:
* [Quantum Error Correction](https://codebook.xanadu.ai/E.1)
* [Noisy circuits](https://pennylane.ai/qml/demos/tutorial_noisy_circuits/)
* [qml.operation.Channel](https://docs.pennylane.ai/en/stable/code/api/pennylane.operation.Channel.html)